In [1]:
import networkx as nx

def get_gc_nodes(graph):
	graphs = list(nx.connected_component_subgraphs(graph))

	if (len(graphs) == 0):
		return []

	gc = max(graphs, key=len)

	gc_nodes = list(gc.nodes)

	return gc_nodes

# ToDo: Fix the implementation of following
def get_gc_size(g1, g2, dependent_nodes):
    
    common_nodes = [node for node in list(g1.nodes) if node in list(g2.nodes)]
    
    remaining_dependent = [node for node in common_nodes if node in dependent_nodes]
    
    if len(remaining_dependent) > 0:
        gc_size = len(get_gc_nodes(g1)) + len(get_gc_nodes(g2))
    
    else:
        gc_size = max(len(get_gc_nodes(g1)), len(get_gc_nodes(g2)))
        
    return gc_size


def get_not_gc_nodes(graph):
    all_nodes = list(graph.nodes)

    gc_nodes = get_gc_nodes(graph)
    
    not_gc_nodes = [node for node in all_nodes if node not in gc_nodes]
    
    return not_gc_nodes


# returns list of gcs
def remove_nodes(g1, g2, nodes_to_remove):
	# calculate gc(g1, g2) and initiate list gcs with this value.
	gcs = [get_gc_size(g1, g2)]

	# for each node n in nodes to remove call remove_nodes(g1, g2, nodes_to_remove) and # store the gc(g1, g2) in the list gcs
	for node in nodes_to_remove:
		remove_node(g1, g2, node)

		gcs.append(get_gc_size(g1, g2))	
	# return gcs
	return gcs

# returns void
def remove_node(g1, g2, node_to_remove):
	# 0. If node_to_remove belong to g1 then following else return
	if (node_to_remove not in list(g1.nodes)):
		return

	# 1. remove adjacent edges of node_to_remove in g1. Initiate list “nodes” with # node_to_remove
	g1.remove_node(node_to_remove)

	removed_nodes = [node_to_remove]

	# 2. remove remove all non-gc nodes in g1 and store the list of these non-gc nodes in “nodes”
	non_gc_nodes = get_not_gc_nodes(g1)

	g1.remove_nodes_from(non_gc_nodes)

	removed_nodes.extend(non_gc_nodes)

	# 3. find the dependent nodes of “removed_nodes” in g2 and store in new_nodes_to_remove
	# Assumption new_nodes_to_remove are same as removed_nodes

	# 4. Call remove_nodes(g2, g1, new_nodes_to_remove)
	remove_nodes(g2, g1, removed_nodes)

def generic_remove_nodes(g1, g2, dependent_nodes, nodes_to_remove):
	gcs = [get_gc_size(g1, g2, dependent_nodes)]

	for node in nodes_to_remove:
		generic_remove_node(g1, g2, dependent_nodes, node)

		gcs.append(get_gc_size(g1, g2, dependent_nodes))

	return gcs

def generic_remove_node(g1, g2, dependent_nodes, node_to_remove):
	if (node_to_remove not in list(g1.nodes)):
		return
	
	g1.remove_node(node_to_remove)

	removed_nodes = [node_to_remove]

	non_gc_nodes = get_not_gc_nodes(g1)

	g1.remove_nodes_from(non_gc_nodes)

	removed_nodes.extend(non_gc_nodes)

	new_nodes_to_remove = [node for node in removed_nodes if node in dependent_nodes]

	generic_remove_nodes(g2, g1, dependent_nodes, new_nodes_to_remove)
    
